In [2]:
import json
from rouge import Rouge
import os
import pandas as pd
import numpy as np
import copy
import jieba
from jieba import posseg
import pickle
import sys
import re
from preprocess_data import *
import logging
import gensim
from gensim.models.word2vec import LineSentence
from sklearn import model_selection
from config import config_base
import vocab
config = config_base.config

In [4]:
df= organize_data(config.train_data)
df = deal_data(df)
df = df.sample(5000)
df = shorten_content_all(df, config.max_len)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.479 seconds.
Prefix dict has been built succesfully.


shorten content, accuracy: 0.9674
max length: 300
min length: 2
mean length:180
median length:175


In [3]:
a.requires_grad

False

In [25]:
df= organize_data(config.train_data)
df = deal_data(df)
df = df.sample(5000)
df = shorten_content_fix(df, config.max_len)

shorten content, accuracy: 0.9618
max length: 300
min length: 2
mean length:189
median length:185


In [22]:
import thulac
thu1 = thulac.thulac(seg_only=True)

Model loaded succeed


In [23]:
def cut(string):
    tmp = thu1.cut(string)
    a, b = zip(*tmp)
    return a

In [24]:
def shorten_content_fix(df, max_len):
    """
    :param df:
    :param max_len:
    :return: df
    """
    sys.setrecursionlimit(1000000)
    rouge = Rouge(metrics=['rouge-l'])

    def match(title, content, question, max_len):

        # 如果无法用'。'划分，直接返回 title
        if '。' not in content:
            return title

        def count(flag, content_list):
            """ 查数 """
            number = 0
            for i in range(len(flag)):
                if flag[i] != 0:
                    number += len(content_list[i])+1
            return number

        # 过滤
        content_list = content.split('。')
        temp = []
        for c in content_list:
            if c not in ['', ' ', '  ']:
                temp.append(c)
        content_list = temp
        content_list = [cut(c) for c in content_list]
        content_len = len(content_list)

        question_str = ' '.join(cut(question))

        # 相似性得分: rouge-l
        scores = []
        for c in content_list:
            if ''.join(c) in question:
                scores.append(-5)
                continue
            c_str = ' '.join(c)
            score = rouge.get_scores(c_str, question_str, avg=True)['rouge-l']['r']
            scores.append(score)

        # 标记类型
        flag = np.zeros(content_len)
        title_number = len(cut(title))
        max_len = max_len - title_number
        flag_result = flag.copy()

        # 核心句:
        max_score = max(scores)
        for i in range(content_len):
            if scores[i] == max_score:
                flag[i] = -1
        number = count(flag, content_list)
        if number <= max_len:
            flag_result = flag.copy()
        else:
            question_set = set(jieba.lcut(question, cut_all=True))
            for j in range(content_len):
                if flag[j] == -1:
                    c = ''.join(content_list[j])
                    c_set = set(jieba.lcut(c, cut_all=True))
                    s = len(c_set & question_set)
                    flag[j] = s
            max_s = max(flag)
            for j in range(content_len):
                if flag[j] == max_s:
                    flag_result[j] = -1

            temp = []
            c = 0
            for j in range(content_len):
                if flag_result[j] == -1:
                    c += len(content_list[j]) + 1
                    if c > max_len:
                        break
                    temp.append(''.join(content_list[j]))
            result = [title] + temp

            return '。'.join(result)

        # 核心句下一句
        if number <= max_len:
            for i in range(content_len):
                if (flag[i] == -1) and (i+1 < content_len) and (flag[i+1] == 0):
                    flag[i+1] = -2
            number = count(flag, content_list)
            if number < max_len:
                flag_result = flag.copy()

        # 最后一句
        if number < max_len:
            if flag[-1] == 0:
                flag[-1] = -3
            number = count(flag, content_list)
            if number < max_len:
                flag_result = flag.copy()

        # 第一句
        if number < max_len:
            if flag[0] == 0:
                flag[0] = -4
            number = count(flag, content_list)
            if number < max_len:
                flag_result = flag.copy()

        # 蕴含句（上+中+下）
        if number < max_len:
            for i in range(content_len):
                if scores[i] == -5:
                    if (i-1 >= 0) and (flag[i-1] == 0):
                        flag[i-1] = -5
                    if flag[i] == 0:
                        flag[i] = -5
                    if (i+1 < content_len) and (flag[i+1] == 0):
                        flag[i+1] = -5
            number = count(flag, content_list)
            if number < max_len:
                flag_result = flag.copy()

        # 核心句下下句
        if number < max_len:
            for i in range(content_len):
                if (flag[i] == -1) and (i+2 < content_len) and (flag[i+2] == 0):
                    flag[i+2] = -6
            number = count(flag, content_list)
            if number < max_len:
                flag_result = flag.copy()

        # 核心句上一句
        if number < max_len:
            for i in range(content_len):
                if (flag[i] == -1) and (i-1 >= 0) and (flag[i-1] == 0):
                    flag[i-1] = -7
            number = count(flag, content_list)
            if number < max_len:
                flag_result = flag.copy()

        # 核心句下下下句
        if number < max_len:
            for i in range(content_len):
                if (flag[i] == -1) and (i+3 < content_len) and (flag[i+3] == 0):
                    flag[i+3] = -8
            number = count(flag, content_list)
            if number < max_len:
                flag_result = flag.copy()

        # 核心句上上句
        if number < max_len:
            for i in range(content_len):
                if (flag[i] == -1) and (i-2 >= 0) and (flag[i-1] == 0):
                    flag[i-2] = -9
            number = count(flag, content_list)
            if number < max_len:
                flag_result = flag.copy()

        result = [title]
        for i in range(content_len):
            if flag_result[i] != 0:
                result.append(''.join(content_list[i]))

        # 过滤重复
        temp = []
        for r in result:
            if r not in temp:
                temp.append(r)
        result = temp

        return '。'.join(result)

    titles = df['title'].values
    contents = df['content'].values
    questions = df['question'].values

    merge = [match(t, c, q, max_len) for t, c, q in zip(titles, contents, questions)]
    df['merge'] = merge

    # 评估数据集构建效果
    if 'answer' in df:
        answers = df['answer'].values
        is_in = [True if a in m else False for m, a in zip(merge, answers)]
        df['is_in'] = is_in
        print('shorten content, accuracy: %.4f' % (sum(is_in)/len(df)))

    merge_len = [len(cut(m)) for m in merge]
    df['len'] = merge_len
    print('max length: %d' % max(merge_len))
    print('min length: %d' % min(merge_len))
    print('mean length:%d' % df['len'].mean())
    print('median length:%d' % df['len'].median())

    return df